In [53]:
from pathlib import Path

import openpyxl
import polars as pl
from nltk.tokenize import TweetTokenizer

# HELPERS

In [54]:
def ruta_parquet(nombre_archivo: str) -> Path:
    ruta = f"data/{nombre_archivo}.parquet"
    rutafinal = Path().parent / ruta
    print(f"Ruta final: {rutafinal}")
    return rutafinal

def cargar_parquet(nombre_parquet: str) -> pl.LazyFrame:
    ruta = ruta_parquet(nombre_parquet) 
    return pl.read_parquet(ruta).lazy()

def guardar_parquet(data: pl.LazyFrame | pl.DataFrame, nombre_parquet: str) -> pl.LazyFrame:
    ruta = ruta_parquet(nombre_parquet)
    if isinstance(data, pl.LazyFrame):
        data.sink_parquet(ruta)
        return cargar_parquet(nombre_parquet)
    else:
        data.write_parquet(ruta)
        return cargar_parquet(nombre_parquet)

In [55]:
IMPRIMIR = True

def imprimir(mensaje: str, parametros=None, collect_lazy_frame=False, **kwargs):
    if not IMPRIMIR:
        return
    # Normaliza la fuente de valores: dict explícito o kwargs
    values = {}
    if isinstance(parametros, dict):
        values.update(parametros)
    values.update(kwargs)

    # Soporte opcional para LazyFrame
    if collect_lazy_frame:
        for k, v in list(values.items()):
            if hasattr(v, "collect"):
                values[k] = v.collect()

    print(mensaje.format(**values))

In [56]:

# DEFINICIONES INICIALES
ruta = Path().parent / "data/opiniones.xlsx"
print(f"Ruta del archivo Excel: {ruta} [exists: {ruta.exists()}]")

workbook = openpyxl.load_workbook(ruta, read_only=True)
hojas = workbook.sheetnames


def obtener_nombre_hoja(sheet_index: int) -> str:
    """Obtiene el nombre de la hoja dado su índice."""
    nombre_hoja = hojas[sheet_index].lower().replace(" ", "_")
    return nombre_hoja


def calificaciones(calificacion:str) -> int:
    """Convierte la calificación de string a int."""
    calificacion_map = {
        "Pésimo": 1,
        "Malo": 2,
        "Regular": 3,
        "Muy bueno": 4,
        "Excelente": 5
    }
    return calificacion_map.get(calificacion, 0)


def tratar_columna(col:str) -> str:
    """Trata el nombre de una columna para que sea válido en polars."""
    col = col.strip().lower().replace(" ", "_").replace("-", "_")
    return col


def añadir_columna_lugar_turistico(iteracion: int) -> pl.LazyFrame:
    df_excel = pl.read_excel(ruta, sheet_id=iteracion+1).lazy()
    nombre_hoja = obtener_nombre_hoja(iteracion)
    
    df_excel = (
        df_excel
        .with_columns([
            pl.lit(nombre_hoja).alias("lugar_turistico"),
            pl.col("Calificación").map_elements(calificaciones).alias("calificacion_numerica"),
        ])
    )

    df_excel = df_excel.rename({col: tratar_columna(col) for col in df_excel.collect_schema().names()})
    return df_excel


def juntar_df(lista_df: list[pl.LazyFrame]) -> pl.LazyFrame:
    nuevo_df_excel = pl.concat(lista_df)
    return nuevo_df_excel




Ruta del archivo Excel: data/opiniones.xlsx [exists: True]


# Limpiezsa y tratamiento del dataset

In [57]:
lista_df: list[pl.LazyFrame] = []
for i, hoja in enumerate(hojas):
    lista_df.append(añadir_columna_lugar_turistico(i))

df_excel = juntar_df(lista_df)

# GUARDAR PARQUET
df_reseñas = guardar_parquet(df_excel, "reseñas_turisticas")


imprimir(f"{df_reseñas.collect()['opinión'][:5]}")
print(f"Numero de opiniones: {df_reseñas.collect().height:,}")
MENSAJE = f" Columnas: {df_reseñas.collect().columns}"
imprimir(F"{"="*len(MENSAJE)}")
imprimir(MENSAJE)
imprimir(F"{"="*len(MENSAJE)}")


Ruta final: data/reseñas_turisticas.parquet
Ruta final: data/reseñas_turisticas.parquet
shape: (5,)
Series: 'opinión' [str]
[
	""Basílica muy bien conservada,…
	""The Basilica (Guanajuato does…
	""Edificio de la iglesia amaril…
	""A must see place in town and …
	""Not particularly impressive, …
]
Numero de opiniones: 10,462
 Columnas: ['género', 'edad', 'nacional_ó_internacional', 'calificación', 'escala', 'número_de_aportaciones', 'título_de_la_opinión', 'opinión', 'país', 'idioma', 'dispositivo', 'fecha', 'lugar_turistico', 'calificacion_numerica']


# CONTRUCCION DEL VOCABULARIO, DISTRIBUCIÓN FRECUENCIAS

In [58]:
class Texto:
    def __init__(self, lazy_frame: pl.LazyFrame, TOP_PALABRAS:int | None = None):
        self.SOS = "<s>"
        self.EOS = "</s>"
        self.UNK = "<unk>"
        self.tokenizer = TweetTokenizer()
        self.lazy_frame = lazy_frame
        self.TOP_PALABRAS = TOP_PALABRAS

        # Guardamos estados
        self._freq_dist: list[tuple[str, int]] = []
        self._final_vocab: set[str] = set()
        self._corpus: list[list[str]] = []
        

    
    def _tokenizar(self, texto: str) -> list[str]:
        tokens = self.tokenizer.tokenize(texto.lower().strip())
        return tokens


    def _tokenizar_data(self) -> pl.LazyFrame:
        df_limpio = (
            self.lazy_frame
            .drop_nulls(subset=["opinión"])
            .with_columns(
                pl.col("opinión")
                .cast(pl.Utf8)
                .str.strip_chars()                                 # recorta espacios
                .str.replace_all(r'^[“”"\'«»\(\)\[\]\s]+', '')      # limpia INICIO
                .str.replace_all(r'[“”"\'«»\(\)\[\]\s]+$', '')      # limpia FINAL
                .alias("opinión_limpia")
            )
            .filter(pl.col("opinión_limpia").str.len_chars() > 0)


            .with_columns(
                pl.col("opinión_limpia")
                .map_elements(self._tokenizar, return_dtype=pl.List(pl.Utf8))
                .alias("opinión_tokenizada")
            )
            .filter(pl.col("opinión_tokenizada").list.len() > 0)
        )
        return df_limpio

    
    def _distribuir_frecuencias(self) -> pl.LazyFrame:
        lf_counts = (
            self._tokenizar_data()
            .select(pl.col("opinión_tokenizada"))
            .explode("opinión_tokenizada")
            .group_by("opinión_tokenizada")
            .len()
            .rename({
                "len": "frecuencia",
                "opinión_tokenizada": "token"
                })
            .sort("frecuencia", descending=True)
        )
        return lf_counts


    

    # ============== VOCABULARIO FINAL ==============
    def _vocabulario_final(self) -> pl.LazyFrame:
        vocab = (
            self._distribuir_frecuencias()
            .select("token")
        )
        return vocab


    
    
    def __build_corpus(self) -> pl.LazyFrame:
        vocab_set = self.vocabulario
        
        PREFIJO_SOS = pl.lit([self.SOS, self.SOS])
        SUFIJO_EOS = pl.lit([self.EOS])
        UNK_LIT = pl.lit(self.UNK)
        VOCAB_LIT = pl.lit(list(vocab_set))

        corpus = (
            self._tokenizar_data()
            .with_row_index("rid")
            .select("rid", "opinión_tokenizada")
            .explode("opinión_tokenizada")
            .with_columns(
                # Aplica el enmascaramiento
                pl.when(pl.col("opinión_tokenizada").is_in(VOCAB_LIT))
                .then(pl.col("opinión_tokenizada"))
                .otherwise(UNK_LIT)
                .alias("tok_mask")
            )
            # Aquí lo que hacemos es juntar todos los tokens (ya enmascarados) en UNA lista por opinión
            .group_by("rid")
            .agg(pl.col("tok_mask").alias("tokens"))
            .with_columns(
                pl.concat_list([PREFIJO_SOS, pl.col("tokens"), SUFIJO_EOS])
                .alias("opinión_trigram")
            )
            .select("opinión_trigram")
        )
        return corpus
    
    
    # ================ PROPIEDADES ==============

    @property
    def freq_dist(self) -> list[tuple[str, int]]:
        if not self._freq_dist:
            self._freq_dist = (
                self._distribuir_frecuencias()
                .collect(engine="streaming")
                .rows()
            )            
        return self._freq_dist
    
    @property
    def vocabulario(self) -> dict:
        if not self._final_vocab:
            df = (
                self._vocabulario_final()
                .select("token")
                .collect(engine="streaming")
                .to_series()
                .to_list()
            )
            self._final_vocab = set(df)
        return self._final_vocab
    

    @property
    def corpus(self) -> list[list[str]]:
        
        if not self._corpus:
            self._corpus = (
                self.__build_corpus()
                .collect(engine="streaming")
                .get_column("opinión_trigram")
                .to_list()
            )
        return self._corpus


## Limpieza
Se esta utilizando una limpieza de codigo de manera aislada y preparada, primero insertando token de inicio y de cierre preparando para poder hacer uso del mismo data set
Lista de cosas:
- Token de inicio y fin de oración

In [59]:
# Asegúrate de tener df_reseñas (LazyFrame) cargado antes de esto.
texto = Texto(df_reseñas)

In [60]:
print(f"{"="*10} CORPUS {"="*10}")
corpus = texto.corpus
print(f"Tamaño del corpus: {len(corpus):,} opiniones")
for opinion in corpus[:5]:
    print(opinion)

========== CORPUS ==========
Tamaño del corpus: 10,462 opiniones
['<s>', '<s>', 'basílica', 'muy', 'bien', 'conservada', ',', 'punto', 'central', 'en', 'la', 'ciudad', ',', 'muy', 'linda', 'por', 'dentro', 'vale', 'la', 'pena', 'darse', 'una', 'vuelta', 'para', 'conocer', ',', 'alto', 'significado', 'religioso', 'y', 'arquitectónico', '.', '</s>']
['<s>', '<s>', 'the', 'basilica', '(', 'guanajuato', 'does', 'not', 'have', 'a', 'cathedral', ',', 'it', 'is', 'situated', 'in', 'the', 'nearby', 'city', 'of', 'leon', ')', 'is', 'truly', 'imposing', 'both', 'during', 'the', 'day', 'and', 'when', 'illuminated', 'at', 'night', '.', 'it', 'also', 'has', 'some', 'lovely', 'polychrome', 'sculpture', 'and', 'a', 'great', 'organ', '.', '</s>']
['<s>', '<s>', 'edificio', 'de', 'la', 'iglesia', 'amarilla', 'es', 'un', 'abigarrado', 'y', 'nahu', 'crear', 'una', 'más', 'bella', 'de', 'atocha', '.', 'he', 'venido', 'a', 'cabo', 'mejor', 'en', 'frente', 'de', 'la', 'misma', 'guanajuato', 'es', 'menor', '

In [61]:
print(f"{"="*10} FREQ_DIST {"="*10}")
freq_dist = texto.freq_dist
print(f"Tamaño de la distribución de frecuencias: {len(freq_dist):,} tokens")
print(freq_dist)

========== FREQ_DIST ==========
Tamaño de la distribución de frecuencias: 23,315 tokens
[('.', 25953), (',', 22328), ('de', 17338), ('la', 13161), ('the', 11691), ('a', 9976), ('y', 9594), ('que', 9502), ('es', 7678), ('el', 7551), ('en', 7183), ('and', 5269), ('un', 5200), ('of', 5064), ('to', 4716), ('no', 4234), ('una', 3898), ('is', 3622), ('para', 3481), ('por', 3464), ('guanajuato', 3456), ('se', 3434), ('los', 3409), ('muy', 3379), ('las', 3130), ('lugar', 2946), ('in', 2904), ('con', 2709), ('del', 2675), ('you', 2548), ('it', 2420), ('lo', 2402), ('ciudad', 2098), ('pero', 1924), ('this', 1893), ('al', 1843), ('hay', 1719), ('si', 1708), ('!', 1704), ('te', 1666), ('i', 1639), ('su', 1555), ('historia', 1549), ('for', 1528), ('este', 1436), ('are', 1424), ('was', 1407), ('como', 1366), ('museo', 1354), ('más', 1344), ('there', 1312), ('but', 1311), ('me', 1216), ('esta', 1161), ('that', 1134), ('with', 1115), ('vista', 1107), ('on', 1103), ('(', 1042), ('o', 1037), ('we', 1002

In [62]:
print(f"{"="*10} VOCABULARIO {"="*10}")
vocabulario = texto.vocabulario
print(f"Tamaño del vocabulario: {len(vocabulario):,}")
print(vocabulario)

========== VOCABULARIO ==========
Tamaño del vocabulario: 23,315
{'books', 'stopping', 'police', 'erected', 'walkway', 'podiamos', 'ético', 'becoming', 'desserts', 'coservación', 'self-regulate', 'storing', 'gobierno', 'políticas', 'dating', 'option', 'regresen', 'japoneses', 'recibes', 'encontrándote', 'repulsión', 'observar', 'versatility', 'opulento', 'hungry', 'janeiro', 'ridículo', 'amarilla', 'guacamole', 'controlan', 'fine', 'callejera', 'romantico', 'creation', 'pinturas', '1972', 'cast', 'relatives', 'quito', 'jarrito', 'cocerts', 'atractivo', 'considerados', 'alojado', 'hesitating', 'reproduciendo', 'ofrecerse', 'peat', 'spainards', 'cola', 'aseo', 'cierto', '17:00', 'sugerencia', 'undoubtedly', 'posing', 'hamburguesa', 'school', 'arrelaron', 'conociste', 'foreign', 'down', 'guerrillas', 'rana', 'caramelo', 'chistoso', 'hacer', 'recepción', 'legado', 'cooked', 'everyone', 'soñado', 'participaste', 'fire', 'inusitado', 'gratificación', 'lagestraemiae', 'tendrá', 'empinado', 'm